|<h2>Course:</h2>|<h1><a href="https://udemy.com/course/dullms_x/?couponCode=202508" target="_blank">A deep understanding of AI language model mechanisms</a></h1>|
|-|:-:|
|<h2>Part 5:</h2>|<h1>Observation (non-causal) mech interp<h1>|
|<h2>Section:</h2>|<h1>Investigating neurons and dimensions<h1>|
|<h2>Lecture:</h2>|<h1><b>CodeChallenge HELPER: Category-tuned MLP projections<b></h1>|

<br>

<h5><b>Teacher:</b> Mike X Cohen, <a href="https://sincxpress.com" target="_blank">sincxpress.com</a></h5>
<h5><b>Course URL:</b> <a href="https://udemy.com/course/dullms_x/?couponCode=202508" target="_blank">udemy.com/course/dullms_x/?couponCode=202508</a></h5>
<i>Using the code without the course may lead to confusion or errors.</i>

In [ ]:
import numpy as np
import scipy.stats as stats

import torch
from transformers import AutoModelForCausalLM, GPT2Tokenizer

import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import matplotlib as mpl
import matplotlib_inline.backend_inline
matplotlib_inline.backend_inline.set_matplotlib_formats('svg')

In [ ]:
# import gpt tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Exercise 1: Tokenize text and find target locations

In [ ]:
# generated by claude
sentences = [

    # sentences with "toothpaste," "toothbrush," "floss," or "mouthwash"

    "She bought a new electric toothbrush that has a timer built in.",
    "I ran out of toothpaste this morning and had to borrow some from my roommate.",
    "After brushing, he used mouthwash to freshen his breath before the interview.",
    "The dentist recommended using dental floss daily to prevent gum disease.",
    "My travel toothbrush folds up to fit easily in my toiletry bag.",
    "The minty toothpaste made my mouth feel clean and refreshed.",
    "He swished the mouthwash around for thirty seconds before spitting it out.",
    "Waxed floss works better for me because it doesn't fray between my teeth.",
    "The hotel provided complimentary toothbrush and toothpaste in the bathroom.",
    "My children prefer fruit-flavored toothpaste over mint varieties.",
    "The dentist gave me a free sample of antibacterial mouthwash after my cleaning.",
    "Learning to floss properly took me several attempts and a lesson from my hygienist.",
    "I keep a spare toothbrush at work for after-lunch brushing.",
    "The natural toothpaste didn't contain fluoride but still cleaned effectively.",
    "Using mouthwash too frequently can disrupt the beneficial bacteria in your mouth.",
    "Water flossers are a good alternative for people who struggle with traditional floss.",
    "The bristles on my toothbrush were frayed, indicating it was time for a replacement.",
    "She squeezed the last bit of toothpaste from the tube by rolling it up from the bottom.",
    "Alcohol-free mouthwash is often recommended for people with sensitive gums.",
    "The dental hygienist demonstrated the correct technique for using floss picks.",

    # sentences with "dishwasher," "nightstand," "bookshelf," or "doorknob"

    "The dishwasher hummed quietly as it ran through its cleaning cycle.",
    "She placed her reading glasses on the nightstand before turning off the lamp.",
    "The antique bookshelf dominated the wall with its impressive collection of leather-bound books.",
    "He turned the doorknob slowly, trying not to wake the sleeping baby in the next room.",
    "Loading the dishwasher efficiently is truly an art form that few have mastered.",
    "The water glass on the nightstand had left a circular mark on the wooden surface.",
    "They installed a new bookshelf to accommodate their growing collection of novels.",
    "The brass doorknob gleamed after being polished with special metal cleaner.",
    "The dishwasher leaked water onto the kitchen floor, creating a small puddle.",
    "Her phone alarm buzzed loudly on the nightstand, jolting her awake.",
    "The bookshelf sagged in the middle from the weight of encyclopedias and textbooks.",
    "The doorknob came loose in his hand when he tried to enter the storage room.",
    "Energy-efficient dishwasher models use significantly less water than older versions.",
    "The small drawer in the nightstand contained a journal and several pens.",
    "The children's favorite picture books were displayed on the lowest shelf of the bookshelf.",
    "The vintage glass doorknob was the original fixture from when the house was built.",
    "He loaded the dirty dishes into the dishwasher while singing along to the radio.",
    "The prescription medication and reading light sat on the nightstand within easy reach.",
    "They secured the tall bookshelf to the wall to prevent it from tipping over.",
    "The decorative doorknob added a touch of elegance to the otherwise plain door."
]

n_sentences = len(sentences)
labels = (np.linspace(0,1,n_sentences)>.5).astype(int)
labels

In [ ]:
targetwords = [' toothpaste',' toothbrush',' floss',    ' mouthwash',
               ' dishwasher',' nightstand',' bookshelf',' doorknob'  ]

targtoks = []

for word in targetwords:
  targtoks.append(tokenizer.encode(word))
  print

targtoks

In [ ]:
# tokenize the sentences
tokenizer.pad_token = tokenizer.eos_token
tokens = tokenizer(, return_tensors='pt', padding)
seq_len =

tokens['input_ids'][0], tokens['attention_mask'][0]

In [ ]:
# (see code file part5_neurons_multitokenWords for more details)

# initialize
targetlocs = np.zeros(n_sentences,dtype=int)
targetvals = np.zeros(n_sentences,dtype=int)


# loop over sentences
for senti in range(n_sentences):

  # loop over target words
  for targi in range(len(targtoks)):

    # how many tokens in this target word?
    targlen = len(targtoks[targi])

    # loop over all tokens in this sequence
    for ti in range(targlen,seq_len+1):

      # target token if the previous mini-sequence matches
      if torch.equal(tokens['input_ids'][senti,ti-targlen:ti],torch.tensor(targtoks[targi])):
        # target index in sequence
        # target value
        print(f"Sentence {} target {} at index {}: '{}'")

In [ ]:
# confirmation
for t in tokens['input_ids'][1]:
  print(

In [ ]:
u,c = np.unique(targetvals,)
for uu,cc in zip(u,c):
  print

# Exercise 2: Forward pass and get target-word activations

In [ ]:
# import the model
model = AutoModelForCausalLM.from_pretrained('gpt2-large')
model.eval()

In [ ]:
# a function to hook the activations
activations = {}

def implant_hook(layer_number):
  def hook(module, input, output):
    activations[f'mlp_{layer_number}'] =
  return hook

# put hooks in all layers
for layer2hook in range(model.config.n_layer):
  .register_forward_hook(implant_hook(layer2hook))

In [ ]:
# process the tokens


In [ ]:
activations.keys(), activations['mlp_4'].shape

In [ ]:
# initialize
targetActs =

# loop over sentences
for senti in range(n_sentences):

  # loop over layers
  for layeri in range(model.config.n_layer):

    # get activations from target embeddings vector
    targetActs[layeri,senti,:] =

targetActs.shape

# Exercise 3: T-tests for dental vs. furniture categories

In [ ]:
# two-samples t-test on all neurons between the two categories
tres = stats.ttest_ind(
    targetActs,
    targetActs, axis=)

# extract t-values and p-values
data1_t =
data1_p =

# print the shape
print(data1_t.shape)

In [ ]:
# significance threshold (p<.05, Bonferroni)
sigPthresh =

# setup the figure
fig = plt.figure(figsize=(14,4))
gs = GridSpec(1,3,figure=fig)
ax1 = fig.add_subplot(gs[:2])
ax2 = fig.add_subplot(gs[2])


# loop over layers
numsig = np.zeros(model.config.n_layer)
for layeri in range(model.config.n_layer):

  # find the significant neurons
  issig =
  numsig[layeri] =

  # draw them
  ax1.plot(,,'rx',alpha=.2,markersize=2)
  ax1.plot(,,'ko',markerfacecolor=mpl.cm.plasma(layeri/model.config.n_layer),markersize=8,alpha=.3)

ax1.set(xlabel='Layer',ylabel='t-statistic',xticks=range(model.config.n_layer),
        xlim=[-1,model.config.n_layer],title='T-values for activations of dental vs. furniture')


# percent significant neurons per layer
ax2.bar()
ax2.set(xlabel='Layer',ylabel='% of neurons per layer',xticks=range(0,model.config.n_layer,3),
        xlim=[-1,model.config.n_layer],title='Percent significant neurons')

plt.tight_layout()
plt.show()

# Exercise 4: Per-target activations in t-max neurons

In [ ]:
# only showing one layer in this plot
layeri = 16

# pick the max-statistic neurons for each category
maxt = data1_t
mint = data1_t


# draw the figure!
plt.figure(figsize=(10,4))
for i in range(targetvals.max()+1):

  # find max activations for this target and plot
  ys =
  plt.plot()

  # find max activations for this target and plot
  ys =
  plt.plot

plt.axhline
plt.axvline
plt.gca().set(xticks=range(targetvals.max()+1),xticklabels=targetwords,ylabel='Activation',
              title=f'Word and category activations in layer {layeri}')
plt.legend([f'L{layeri} / #{maxt}',f'L{layeri} / #{mint}'])

plt.show()

# Exercise 5: Tokenize multitarget test sentences

In [ ]:
# test sentences with both dental and furniture words
sentences_mix = [
    "She left her toothbrush on the nightstand after falling asleep reading.",
    "The tube of toothpaste rolled off the bookshelf and landed on the floor.",
    "He gargled mouthwash while waiting for the dishwasher cycle to finish.",
    "I keep dental floss in the small drawer of my nightstand for convenient access.",
    "The toothpaste cap fell behind the bookshelf, making it difficult to retrieve.",
    "Her electric toothbrush charging base sits next to the doorknob on the bathroom counter.",
    "After using mouthwash, he placed the cap back on and returned it to the bookshelf.",
    "The floss container was knocked off the nightstand by the cat during the night.",
    "He squeezed the last bit of toothpaste from the tube before placing it in the dishwasher's utensil basket.",
    "The mouthwash spilled when he bumped into the doorknob while carrying too many items.",
    "She organized her bathroom supplies, placing her toothbrush and floss in a basket on top of the bookshelf.",
    "The minty smell of toothpaste lingered near the nightstand where she had been brushing her teeth.",
    "He accidentally splashed mouthwash on the dishwasher while rinsing his mouth.",
    "The children's floss picks were stored in a decorative jar on the bookshelf.",
    "Her toothbrush fell from her hand as she reached for the doorknob in a hurry.",
    "The blue toothpaste stained the wood of the nightstand when the cap wasn't secured properly.",
    "He kept a travel-sized mouthwash in the drawer of his nightstand for freshening up in the morning.",
    "The floss got tangled around the doorknob when the package fell and unraveled.",
    "She placed her toothbrush in the dishwasher's silverware basket to sanitize it thoroughly.",
    "The bathroom renovation included a built-in bookshelf with designated spots for toothpaste, mouthwash, and floss."
]

In [ ]:
tokens_mix =
seq_len_mix =
n_sentences_mix = len(sentences_mix)

In [ ]:
tokens_mix['input_ids'][0]

In [ ]:
targetvals_mat = np.full((n_sentences_mix,seq_len_mix),-1,dtype=int)

# loop over sentences
for senti in range(n_sentences_mix):

  # loop over targets
  for targi in range(len(targtoks)):

    # get target length (number of tokens)
    targlen = len(targtoks[targi])

    # loop over tokens in sentence
    for ti in range(targlen,seq_len_mix+1):

      # if this mini-sequence matches the target
      if torch.equal(tokens_mix['input_ids'][senti,ti-targlen:ti],torch.tensor(targtoks[targi])):
        targetvals_mat
        print(f"Sentence {} target {} at index {}: '{}'")

In [ ]:
u,c = np.unique(,return_counts=True)
for uu,cc in zip(u,c):

In [ ]:
fig,ax = plt.subplots(1,figsize=(10,4))

# Note: The colormap in the video only contains 8 possible colors, so 6&7 were mapped to the same color.
#       The code here correctly assigns a unique color to each label (and -1).
cm_name = 'Set1_r'

ax.imshow(,aspect='auto',cmap=cm_name,vmin=-2,vmax=8)


# discrete colorbar
cmap = mpl.colormaps[cm_name]
norm = mpl.colors.BoundaryNorm(np.arange(-1,9), cmap.N)
sm = mpl.cm.ScalarMappable(cmap=cmap, norm=norm)
cbar = fig.colorbar(sm,ax=ax,pad=.01,label='Target index')

plt.show()

In [ ]:
# push through the model
with torch.no_grad(): model(**tokens_mix)

In [ ]:
print('Activation names:',activations.keys(),'\n')
print('Activation matrix size:',activations['mlp_4'].shape,'\n')

# Exercise 6: T-tests on test data

In [ ]:
# this cell takes ~2.5 mins

# initialize
data2_t = np.zeros
data2_p = np.zeros

# loop over layers
for layeri in range(model.config.n_layer):

  # loop over neurons
  for neuri in range(model.config.n_embd):

    # get the activations
    tmpa = activations[f'mlp_{layeri}'][:,:,neuri]

    # t-test and store the results
    ttmp = stats.ttest_ind
    data2_t[layeri,neuri] =
    data2_p[layeri,neuri] =


In [ ]:
# vectorize for convenience
data1_t_vector =
data1_p_vector =
data2_t_vector =
data2_p_vector =

# find where one or both are significant
bothSig =


# "concordancy" as same sign and at least one is significant
concordance =

# compare the two sets
plt.plot(data1_t_vector,data2_t_vector,'rx',markersize=3,alpha=.1,label='Neither sig')
plt.plot(data1_t_vector,data2_t_vector,'ko',markerfacecolor=[.9,.7,.7],markersize=5,alpha=.4,label='One sig.')
plt.plot(data1_t_vector,data2_t_vector,'ks',markerfacecolor=[.7,.9,.7],markersize=8,label='Both sig.')
plt.legend()

plt.show()
# aka the pistachio canoli plot :P